In [3]:
import lightgbm as lgb
from sklearn.metrics import mean_squared_error, accuracy_score
from sklearn.model_selection import train_test_split

In [ ]:
def build_lag_features(df, feature_cols, target_col, window_size=30, horizon=10):
    X, y = [], []
    for i in range(window_size, len(df) - horizon):
        features = []
        for col in feature_cols:
            lagged = df[col].iloc[i - window_size:i].values
            features.extend(lagged)
        X.append(features)
        y.append(df[target_col].iloc[i + horizon])
    return np.array(X), np.array(y)

In [ ]:
# build lag matrix
X_lag, y_lag = build_lag_features(df, feature_cols, target_col="adc", window_size=30, horizon=10)

# train/test split
X_train, X_test, y_train, y_test = train_test_split(X_lag, y_lag, test_size=0.2, shuffle=False)

# модель
model = lgb.LGBMRegressor(n_estimators=200, learning_rate=0.05)
model.fit(X_train, y_train)

In [ ]:
# прогноз
y_pred = model.predict(X_test)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
sign_acc = accuracy_score(y_test > 0, y_pred > 0)

print(f"RMSE: {rmse:.4f}")
print(f"Sign Accuracy: {sign_acc:.4f}")

In [ ]:
# --- Визуализация ---
plt.figure(figsize=(12, 5))
plt.plot(y_test[:10000], label="True ADC")
plt.plot(y_pred[:10000], label="Predicted ADC")
plt.axhline(0, color='gray', linestyle='--', linewidth=0.5)
plt.title("LSTM Forecast vs True ADC")
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()